# Great Lakes Map with ipyleaflet

In [1]:
# Set up data

import pandas as pd
import matplotlib.pyplot as plt

station_info = pd.read_excel (r'C:\Users\april\Documents\Data Viz\final_project\Great Lakes Basin Model Validations.xlsx')
# replace the space in column titles with underscore
station_info.columns = [c.replace(' (dr)', '_dr') for c in station_info.columns]
station_info.columns = [c.replace(' ', '_') for c in station_info.columns]
station_info.columns = [c.replace('(', '') for c in station_info.columns]
station_info.columns = [c.replace(')', '') for c in station_info.columns]
station_info.columns = [c.replace('Calibration Station', 'Calibration_Station') for c in station_info.columns]

lats = list(station_info.Latitude)
lons = list(station_info.Longitude)
streamflow_dr = list(station_info.Streamflow_dr)
water_temp_dr = list(station_info.Water_Temperature_dr)
sus_sed_dr = list(station_info.Suspended_Sediment_Load_dr)
water_temp_mae = list(station_info.Water_Temperature_mae)
station_num = list(station_info.Calibration_Station)

station_info.head()

,Calibration_Station,Latitude,Longitude,Streamflow_dr,Water_Temperature_dr,Suspended_Sediment_Load_dr,Water_Temperature_mae
0,1,49.597992,-87.964216,0.391237,NaN,NaN,NaN
1,2,48.926076,-87.689935,0.509740,NaN,NaN,NaN
2,3,48.904724,-88.375945,0.407360,NaN,NaN,NaN
3,4,48.848033,-86.606977,0.456780,NaN,NaN,NaN
4,5,48.822786,-88.531901,0.511682,NaN,NaN,NaN


In [2]:
# Make map

from ipyleaflet import Map, Marker, CircleMarker, Popup, FullScreenControl# May need to run jupyter notebookas administrator to get maps to work.
from ipywidgets import HTML
import numpy as np
from colormap import rgb2hex      # (pip install colormap easydev)





# Create colormap in RGB
colormap = plt.get_cmap('autumn')
#sf_dr_array_norm255 = np.round(((np.array(streamflow_dr)+1) * (255/2)),0).astype(int) # Use this code if dr often has negative values.
sf_dr_array_norm255 = np.round(((np.array(streamflow_dr)) * (255)),0).astype(int)
colors_rgb = colormap(sf_dr_array_norm255)

# Make Map
center = (46, -83)
m = Map(center=center, zoom=5)
          
def html_table(index):
    message = """<html>
    <head>Station Information</head>
    <body></body>
    <table border="1">
        <tr>
            <th scope = "row">Calibration Station</th>
            <td> #{} </td>
        </tr>
        <tr>
            <th scope = "row">Coordinates</th>
            <td> {} </td>
        </tr>
        <tr>
            <th scope = "row">Streamflow dr</th>
            <td> {} </td>
        </tr>
        <tr>
            <th scope = "row">Water Temperature dr</th>
            <td> {} </td>
        </tr>
        <tr>
            <th scope = "row">Suspended Sediment Load dr</th>
            <td> {} </td>
        </tr>
        <tr>
            <th scope = "row">Water Temperature mae</th>
            <td> {} </td>
        </tr>
    </table>
    </html>
    """.format(index, 
               (round(lats[index],2), round(lons[index], 2)),
               round(streamflow_dr[index], 6),
               round(water_temp_dr[index], 6),
               round(sus_sed_dr[index], 6),
               round(water_temp_mae[index], 6)
               
              )
    return message



# Add points
for station in station_info.Calibration_Station:
    circle_marker = CircleMarker()
    circle_marker.location = (float(lats[station]), float(lons[station]))
    circle_marker.radius = 3

    # Convert RGB to Hex, then color circles
    colors_hex = rgb2hex(int(colors_rgb[(station-1),0]*255), 
                         int(colors_rgb[(station-1),1]*255), 
                         int(colors_rgb[(station-1),2]*255))

    
    # 

    circle_marker.color = colors_hex
    circle_marker.fill_color = colors_hex
    m.add_layer(circle_marker)
    
    message = HTML()
    message.value = html_table(station)
    message.palceholder = "placeholder"
    #message.description = "Station Information:"
    circle_marker.popup = message
    


In [3]:
from ipyleaflet import Map, basemaps, WidgetControl
from ipywidgets import IntSlider, ColorPicker, jslink
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import clear_output

station_picker = widgets.IntText(
    value=174,
    description='Station #:',
    disabled=False
)

widget_control = WidgetControl(widget=station_picker, position='bottomright')
m.add_control(widget_control)
m.add_control(FullScreenControl())

In [31]:
m

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

In [32]:


def f(x):
    clear_output()
    message_info = HTML()
    message_info.value = html_table(x)
    html_table(x)
    user_select_popup = Popup(
        location = ((float(lats[x]), float(lons[x]))),
        child = message_info
    )
    m.add_layer(user_select_popup)
    return 0



interact(f, x=station_picker) 


m

interactive(children=(IntText(value=174, description='Station #:'), Output()), _dom_classes=('widget-interact'…

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …